In [1]:
import pandas as pd
import numpy as np
import urllib.request
import itertools
import math

from sqlalchemy import create_engine
nyc_database = create_engine('sqlite:///nyc_database.db')

In [4]:
df = pd.read_csv("fhv_tripdata_2019-01.csv", nrows=10)
df.head()

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,B00001,2019-01-01 00:30:00,2019-01-01 02:51:55,NaN,NaN,NaN
1,B00001,2019-01-01 00:45:00,2019-01-01 00:54:49,NaN,NaN,NaN
2,B00001,2019-01-01 00:15:00,2019-01-01 00:54:52,NaN,NaN,NaN
3,B00008,2019-01-01 00:19:00,2019-01-01 00:39:00,NaN,NaN,NaN
4,B00008,2019-01-01 00:27:00,2019-01-01 00:37:00,NaN,NaN,NaN


In [5]:
df['PULocationID']

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: PULocationID, dtype: float64

In [6]:
j, chunksize = 1, 100000
for month in range(1,7):
    fp = "fhv_tripdata_2019-{0:0=2d}.csv".format(month)
    for df in pd.read_csv(fp, chunksize=chunksize, iterator=True):
        df = df.rename(columns={c: c.replace(' ', '_') for c in df.columns})
        df['pickup_hour'] = [x[11:13] for x in df['pickup_datetime']]
        df['dropoff_hour'] = [x[11:13] for x in df['dropoff_datetime']]
        df.index += j
        df.to_sql('table_record', nyc_database, if_exists='append')
        j = df.index[-1] + 1
del df

In [18]:
df_do = pd.read_sql_query('SELECT DOLocationID AS DOLocationID, count(*) AS DOcount, PULocationID AS PULocationID \
                        FROM table_record \
                        WHERE DOLocationID = 34 \
                        GROUP BY DOLocationID, PULocationID', nyc_database)

In [38]:
df_count = pd.read_sql_query('SELECT COUNT(PULocationID) \
                        FROM table_record', nyc_database)
print(df_count)

   COUNT(PULocationID)
0             30264661
   COUNT(PULocationID)
0             30264661


In [19]:
print(df_do)

     DOLocationID  DOcount  PULocationID
0            34.0        1           NaN
1            34.0      110           4.0
2            34.0        1           6.0
3            34.0      135           7.0
4            34.0        1           9.0
5            34.0        6          10.0
6            34.0        4          11.0
7            34.0        2          12.0
8            34.0       50          13.0
9            34.0       65          14.0
10           34.0        1          15.0
11           34.0        3          16.0
12           34.0      979          17.0
13           34.0        6          18.0
14           34.0       19          21.0
15           34.0       13          22.0
16           34.0       24          24.0
17           34.0      325          25.0
18           34.0       53          26.0
19           34.0        4          28.0
20           34.0        1          30.0
21           34.0      329          33.0
22           34.0       78          34.0
23           34.

In [21]:
df_do.to_csv("Navy_Yard_Dropoffs.csv")